In [18]:
import boto3
import pandas as pd


In [5]:
logs_client = boto3.client('logs', region_name='us-west-2')
                        
log_group_name= 'pywren.standalone'

In [27]:
def aws_ts_to_datetime(x):
    """
    aws timestamps are ms past epoch
    pandas / numpy are ns since epochb
    """
    return pd.to_datetime(x*1e6)


In [40]:
# list the log streams

log_stream_prefix = ""

kargs = {'orderBy' : 'LastEventTime', 'descending' : True}

log_streams = []

# Stop at logs whose last event date was later than this
stop_at_datetime = pd.to_datetime("2017-09-10")

paginator = logs_client.get_paginator('describe_log_streams')
log_stream_info = []

done = False

for response in paginator.paginate(logGroupName=log_group_name, **kargs):
    log_stream_info += response['logStreams']
    
    for r in response['logStreams']:
        if aws_ts_to_datetime(r['creationTime']) < stop_at_datetime:
            done = True
    if done:
        break

In [47]:
logstreams_df = pd.DataFrame(log_stream_info)
logstreams_df['instance_id'] = logstreams_df.logStreamName.apply(lambda x : x.split("-")[1])
logstreams_df['logfilename'] = logstreams_df.logStreamName.apply(lambda x : x.split("-")[2])

print("there are", len(logstreams_df), "logstreams")
print("there are", len(logstreams_df.instance_id.unique()), 'instances')
print("there are", len(logstreams_df.instance_id.unique()), 'instances')
print("there are", len(logstreams_df.logfilename.unique()), 'filenames')


there are 1150 logstreams
there are 213 instances
there are 213 instances
there are 6 filenames


In [50]:
def get_all_events(log_group_name, log_stream_name):
    resp = logs_client.get_log_events(logGroupName=log_group_name, 
                                      logStreamName=log_stream_name)
    token_dir = 'nextBackwardToken'
    events = []
    events += resp['events']
    while token_dir in resp:
        resp = logs_client.get_log_events(logGroupName=log_group_name, 
                                    logStreamName=log_stream_name,  
                                   nextToken=resp[token_dir])
        events += resp['events']
        if len(resp['events']) == 0:
            break
    return events

        

In [55]:
tgt_stream_row = logstreams_df.iloc[0]

stream_events = get_all_events(log_group_name, tgt_stream_row.logStreamName)
stream_df = pd.DataFrame(stream_events)
stream_df['timestamp'] = aws_ts_to_datetime(stream_df['timestamp'])
stream_df['ingestionTime'] = aws_ts_to_datetime(stream_df['ingestionTime'])

stream_df.head()

,ingestionTime,message,timestamp
0,2017-09-12 21:05:59.006000128,"2017-09-12 21:05:39,727 CRIT Set uid to user 500",2017-09-12 21:05:50.236000000
1,2017-09-12 21:05:59.006000128,"2017-09-12 21:05:39,727 WARN No file matches v...",2017-09-12 21:05:50.236999936
2,2017-09-12 21:05:59.006000128,"2017-09-12 21:05:39,730 INFO daemonizing the s...",2017-09-12 21:05:50.236999936
3,2017-09-12 21:05:59.006000128,"2017-09-12 21:05:39,730 INFO set current direc...",2017-09-12 21:05:50.236999936
4,2017-09-12 21:05:59.006000128,"2017-09-12 21:05:39,730 INFO supervisord start...",2017-09-12 21:05:50.236999936
